In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "12bbaa9d-ef6e-4adf-a7b9-5c31fb6721a1",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="eazzy_scope",key="Web-App-Secret"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/f38b3e6e-793c-49f9-8207-1087ae98b59b/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://storageanalytics@eazzystrgacc.dfs.core.windows.net/analytics",
  mount_point = "/mnt/analytics/",
  extra_configs = configs)


True

In [0]:
dbutils.fs.ls('/mnt/analytics/')

[FileInfo(path='dbfs:/mnt/analytics/bronze/', name='bronze/', size=0, modificationTime=1759021953000),
 FileInfo(path='dbfs:/mnt/analytics/gold/', name='gold/', size=0, modificationTime=1759021968000),
 FileInfo(path='dbfs:/mnt/analytics/silver/', name='silver/', size=0, modificationTime=1759021962000)]

In [0]:
dbutils.fs.ls('/mnt/analytics/bronze')

[FileInfo(path='dbfs:/mnt/analytics/bronze/customer.parquet', name='customer.parquet', size=3247, modificationTime=1759022138000),
 FileInfo(path='dbfs:/mnt/analytics/bronze/products.parquet', name='products.parquet', size=1239, modificationTime=1759022138000),
 FileInfo(path='dbfs:/mnt/analytics/bronze/shop.parquet', name='shop.parquet', size=867, modificationTime=1759022138000),
 FileInfo(path='dbfs:/mnt/analytics/bronze/transactions.parquet', name='transactions.parquet', size=1757, modificationTime=1759022138000)]

In [0]:
display(dbutils.fs.ls('/mnt/analytics/bronze'))

path,name,size,modificationTime
dbfs:/mnt/analytics/bronze/customer.parquet,customer.parquet,3247,1759022138000
dbfs:/mnt/analytics/bronze/products.parquet,products.parquet,1239,1759022138000
dbfs:/mnt/analytics/bronze/shop.parquet,shop.parquet,867,1759022138000
dbfs:/mnt/analytics/bronze/transactions.parquet,transactions.parquet,1757,1759022138000


In [0]:
# Read the bronze layer
# Read raw data from Bronze layer
df_transactions = spark.read.parquet('/mnt/analytics/bronze/transactions.parquet')
df_products = spark.read.parquet('/mnt/analytics/bronze/products.parquet')
df_shop = spark.read.parquet('/mnt/analytics/bronze/shop.parquet') 
df_customers = spark.read.parquet('/mnt/analytics/bronze/customer.parquet')
display(df_transactions)

transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17
6,110,7,3,5,2025-01-04
7,110,7,2,5,2025-01-01
8,126,7,5,2,2025-06-08
9,123,1,3,2,2024-10-08
10,124,2,2,5,2024-08-27


In [0]:

# STEP 4: CHECK THE DATAFRAME

display(df_shop)

store_id,store_name,location
1,City Mall Store,Mumbai
2,High Street Store,Delhi
3,Tech World Outlet,Bangalore
4,Downtown Mini Store,Pune
5,Mega Plaza,Chennai


In [0]:
# STEP 5: START CLEANING

# Create silver layer - data cleaning

from pyspark.sql.functions import col

# Convert types and clean data
df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

df_shop = df_shop.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers.select(
    "customer_id", "first_name", "last_name", "email", "city", "registration_date"
).dropDuplicates(["customer_id"])

In [0]:
# Join all data together

df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_shop, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))

In [0]:
# VALIDATE THE SILVER DATAFRAMES

display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,Mumbai,2023-07-17,Water Bottle,Fitness,299.0,High Street Store,Delhi,598.0
2,7,110,7,5,2025-01-01,Deepak,Nair,user110@example.com,Mumbai,2023-10-14,Smartwatch,Electronics,4999.0,High Street Store,Delhi,24995.0
4,9,119,29,2,2025-06-03,Kunal,Nair,user119@example.com,Bangalore,2023-08-10,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,8,116,30,5,2025-03-16,Rakesh,Kapoor,user116@example.com,Kolkata,2023-06-15,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1995.0
2,2,124,10,5,2024-08-27,Kavita,Sharma,user124@example.com,Kolkata,2023-11-15,Bluetooth Speaker,Electronics,1299.49,High Street Store,Delhi,6497.45
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95


In [0]:
# STEP 7: COPY TO THE SILVER CONTAINER

# Copy to adls locatioN
silver_path = "/mnt/storageanalytics/silver"

df_silver.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
dbutils.fs.ls('/mnt/analytics/silver')

[FileInfo(path='dbfs:/mnt/analytics/silver/_delta_log/', name='_delta_log/', size=0, modificationTime=1759022533000),
 FileInfo(path='dbfs:/mnt/analytics/silver/part-00000-4fe946d4-4b32-4902-a629-6403417a7406.c000.snappy.parquet', name='part-00000-4fe946d4-4b32-4902-a629-6403417a7406.c000.snappy.parquet', size=6588, modificationTime=1759023054000)]

In [0]:
%sql
CREATE TABLE store_silver_cleaned
USING DELTA
LOCATION 'abfss://storageanalytics@eazzystrgacc.dfs.core.windows.net/analytics/silver/'

In [0]:
%sql
--# STEP 8: Use SQL to validate the data

--# MAGIC %sql 
select * from store_silver_cleaned

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14,Smartwatch,Electronics,4999.00,Mega Plaza,Chennai,14997.00
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,9876543210,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.97
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.00,Tech World Outlet,Bangalore,1996.00
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,9843210987,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.00,Tech World Outlet,Bangalore,7996.00
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,9810987654,Kolkata,2023-08-19,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,596.00
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01,Desk Organizer,Accessories,399.00,Mega Plaza,Chennai,1995.00
2,7,110,7,5,2025-01-01,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14,Smartwatch,Electronics,4999.00,High Street Store,Delhi,24995.00
4,8,116,30,5,2025-03-16,Rakesh,Kapoor,user116@example.com,9732109876,Kolkata,2023-06-15,Desk Organizer,Accessories,399.00,Downtown Mini Store,Pune,1995.00


In [0]:
# STEP 9: CREATE A NEW DATAFRAME IN PREPARATION FOR THE GOLD LAYER


# Load cleaned transactions from Silver layer
silver_df = spark.read.format("delta").load("/mnt/analytics/silver/")


In [0]:
# VALIDATE THE NEW DATAFRAME

display(silver_df)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14,Smartwatch,Electronics,4999.00,Mega Plaza,Chennai,14997.00
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,9876543210,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.97
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.00,Tech World Outlet,Bangalore,1996.00
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,9843210987,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.00,Tech World Outlet,Bangalore,7996.00
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,9810987654,Kolkata,2023-08-19,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,596.00
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01,Desk Organizer,Accessories,399.00,Mega Plaza,Chennai,1995.00
2,7,110,7,5,2025-01-01,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14,Smartwatch,Electronics,4999.00,High Street Store,Delhi,24995.00
4,8,116,30,5,2025-03-16,Rakesh,Kapoor,user116@example.com,9732109876,Kolkata,2023-06-15,Desk Organizer,Accessories,399.00,Downtown Mini Store,Pune,1995.00


In [0]:
# CREATE THE GOLD DATAFRAME BY AGGREEGATING THE SILVER DATAFRAME ACCORDING TO THE REQUIREMENT

from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id", "product_name", "category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)


In [0]:
# VALIDATE THE GOLD DATAFRAME

display(gold_df)


gold_path = "/mnt/analytics/gold/"

gold_df.write.mode("overwrite").format("delta").save(gold_path)

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.00,1,399.000000
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.980000
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.00,1,1995.000000
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.00,1,1996.000000
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.00,1,596.000000
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.990000
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.950000
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.97,1,2399.970000
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.00,1,3998.000000
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.450000


In [0]:
%sql
CREATE TABLE store_gold_sales_summary
USING DELTA
LOCATION 'abfss://storageanalytics@eazzystrgacc.dfs.core.windows.net/analytics/gold/'


In [0]:
%sql
--# VALIDATE THE DATA USING SQL
select * from store_gold_sales_summary


transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.00,1,399.000000
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.980000
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.00,1,1995.000000
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.00,1,1996.000000
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.00,1,596.000000
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.990000
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.950000
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.97,1,2399.970000
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.00,1,3998.000000
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.450000
